In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.comdocker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
final_dataset=df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_dataset['Current_year']=2023

In [ ]:
final_dataset['no_year']=final_dataset['Current_year']-final_dataset['Year']

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Year'],axis=1,inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Current_year'],axis=1,inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset=pd.get_dummies(final_dataset,drop_first=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.corr()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
import matplotlib.pyplot as plt
corrmat=final_dataset.corr()
top_corr_features=corrmat.index
plt.figure(figsize=(20,20))

In [ ]:
sns.heatmap(final_dataset[top_corr_features].corr(),annot=True)

In [ ]:
final_dataset.head()

In [ ]:
X=final_dataset.iloc[:,1:]
y=final_dataset.iloc[:,0]

In [ ]:
X.head()

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model=ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
feat_imprtance=pd.Series(model.feature_importances_,index=X.columns)
feat_imprtance.nlargest(5).plot(kind="barh")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_random=RandomForestRegressor()

In [ ]:
n_estimaters=[int(x) for x in np.linspace(start=100,stop=1200,num=12)]
print(n_estimaters)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
rf=RandomForestRegressor()

In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)


In [ ]:
rf_random.fit(X_train,y_train)


In [ ]:
rf_random.best_params_


In [ ]:
rf_random.best_score_


In [ ]:
predictions=rf_random.predict(X_test)


In [ ]:
sns.distplot(y_test-predictions)


In [ ]:
plt.scatter(y_test,predictions)


In [ ]:
from sklearn import metrics


In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('random_forest_regression_model.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)